# Imports

In [15]:
!pip install easypip

from easypip import easyimport
import functools
import time

easyimport("importlib_metadata==4.13.0")
OmegaConf = easyimport("omegaconf").OmegaConf
bbrl_gym = easyimport("bbrl_gym")
bbrl = easyimport("bbrl>=0.1.6")

import os
import copy
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

import gym
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class
from bbrl.workspace import Workspace
from bbrl.agents import Agents, RemoteAgent, TemporalAgent
from bbrl.agents.gymb import AutoResetGymAgent, NoAutoResetGymAgent
from bbrl.visu.play import load_agent, play
from bbrl.utils.replay_buffer import ReplayBuffer

!git clone https://github.com/EmbersArc/gym-rocketlander
!pip install -e ./gym-rocketlander

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'gym-rocketlander'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
Unpacking objects: 100% (13/13), done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 11
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gym-rocketlander
  Running setup.py develop for gym-rocketlander


# RocketLander

In [13]:
class RocketLanderWrapper(gym.Wrapper):
    """
    Specific wrapper to shape the reward of the rocket lander environment
    """
    def __init__(self, env):
        super(RocketLanderWrapper, self).__init__(env)
        self.env = env
        self.prev_shaping = None
        
    def reset(self):
        self.prev_shaping = None
        return self.env.reset()

    def step(self, action):
        d = 1
        next_state, reward, done, info = self.env.step(action)
        # reward shaping
        """
        shaping = -0.5 * (self.env.distance + self.env.speed + abs(self.env.angle) ** 2)
        shaping += 0.1 * (
            self.env.legs[0].ground_contact + self.env.legs[1].ground_contact
        )
        if self.prev_shaping is not None:
            reward += shaping - self.prev_shaping
        self.prev_shaping = shaping
        """
        # print ("distance", self.env.distance)
        
        # shaping = 0.02
        shaping = 0.008 * (1 - self.env.distance)
        # shaping = 0.1 * (self.env.groundcontact - self.env.speed)
        if (
            self.env.legs[0].ground_contact > 0
            and self.env.legs[1].ground_contact > 0
            and self.env.speed < 0.1
        ):
            d = d * 2
            print("landed !")
            print ("speed", self.env.speed)
            shaping += 6.0 * d / self.env.speed
        else:
          d = 1
        reward += shaping
        return next_state, reward, done, info

    def old_step(self, action):
        next_state, reward, done, info = self.env.step(action)
        # reward shaping
        # shaping = -0.5 * (self.env.distance + self.env.speed + abs(self.env.angle) ** 2)
        # shaping += 0.1 * (self.env.legs[0].ground_contact + self.env.legs[1].ground_contact)
        shaping = 0
        if self.prev_shaping is not None:
            reward += shaping - self.prev_shaping
        self.prev_shaping = shaping

        return next_state, reward, done, info



class FrameSkip(gym.Wrapper):
    """
    Return only every ``skip``-th frame (frameskipping)
    :param env: the environment
    :param skip: number of ``skip``-th frame
    """

    def __init__(self, env: gym.Env, skip: int = 1):
        super().__init__(env)
        self._skip = skip

    def step(self, action: np.ndarray):
        """
        Step the environment with the given action
        Repeat action, sum reward, and max over last observations.
        :param action: the action
        :return: observation, reward, done, information
        """
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break

        return obs, total_reward, done, info

    def reset(self):
        return self.env.reset()

In [25]:
rocket_env = gym.make("RocketLander-v0")
rocket_env = RocketLanderWrapper(rocket_env)
rocket_env = FrameSkip(rocket_env, skip=1)

# SAC

# ROCKETLANDER / SAC